## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/78-googleq-a-train-glove-2lstm-avgv2/model.h5'
tokenizer_path = '/kaggle/input/78-googleq-a-train-glove-2lstm-avgv2/tokenizer.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())

# Model parameters

In [5]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 20000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 200
ANSWER_MAX_LEN = 200

## Test set

In [6]:
tokenizer = load(tokenizer_path)

# Test features
# X_test_title = test['question_title']
# X_test_body = test['question_body']
# X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_seq = test['question_body_uncased']
X_test_answer_seq = test['answer_uncased']

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
X_test_body_seq = tokenizer.texts_to_sequences(X_test_body_seq)
X_test_answer_seq = tokenizer.texts_to_sequences(X_test_answer_seq)

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
X_test_body_seq = pad_sequences(X_test_body_seq, maxlen=BODY_MAX_LEN)
X_test_answer_seq = pad_sequences(X_test_answer_seq, maxlen=ANSWER_MAX_LEN)

X_test = [X_test_title_seq, X_test_body_seq, X_test_answer_seq]

# Model

In [7]:
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

def encoder_block(input_layer):
    encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
    return encoder

## Load embedding

In [8]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
print('n unknown words (GloVe): ', len(glove_unknown_words))

n unknown words (GloVe):  3306


In [9]:
# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title = LSTM(512, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(title)
title = LSTM(256, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(title)
title_out = GlobalAveragePooling1D()(title)

# Body sequence
input_body_seq = Input(shape=(BODY_MAX_LEN,), dtype=tf.float32, name='input_body_seq')
body = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_body_seq)
body = LSTM(512, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(body)
body = LSTM(256, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(body)
body_out = GlobalAveragePooling1D()(body)

# Answer sequence
input_answer_seq = Input(shape=(ANSWER_MAX_LEN,), dtype=tf.float32, name='input_answer_seq')
answer = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_answer_seq)
answer = LSTM(512, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(answer)
answer = LSTM(256, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(answer)
answer_out = GlobalAveragePooling1D()(answer)


# Output
x = Concatenate()([title_out, body_out, answer_out])
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_initializer='lecun_normal')(x)
x = Dropout(0.5)(x)
output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='lecun_normal', name='output')(x)
model = Model(inputs=[input_title_seq, input_body_seq, input_answer_seq], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title_seq (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_body_seq (InputLayer)     [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 300)      17424600    input_title_seq[0][0]            
______________________________________________________________________________________________

# Make predictions

In [10]:
Y_test = model.predict(X_test)

In [11]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.915013,0.593609,0.086590,0.665256,0.761295,0.751605,0.633944,0.573284,0.371557,...,0.880683,0.896223,0.594546,0.960029,0.960838,0.796325,0.182454,0.072155,0.572666,0.910298
1,46,0.886998,0.554867,0.006843,0.719055,0.764358,0.880835,0.550018,0.462107,0.123737,...,0.749212,0.925031,0.628337,0.962960,0.968159,0.840522,0.863268,0.154771,0.126434,0.880270
2,70,0.907012,0.677478,0.024590,0.745364,0.944681,0.914696,0.602402,0.537459,0.418991,...,0.885018,0.899448,0.617597,0.950076,0.957223,0.817313,0.074183,0.061950,0.685571,0.900184
3,132,0.873127,0.491936,0.002857,0.804151,0.814499,0.946146,0.541817,0.425467,0.140534,...,0.741977,0.963939,0.691483,0.980617,0.990351,0.913528,0.754348,0.134878,0.721599,0.920241
4,200,0.899649,0.503609,0.027867,0.805547,0.805958,0.862124,0.585111,0.513471,0.242008,...,0.752472,0.922174,0.656462,0.963941,0.966445,0.846378,0.245598,0.088157,0.584843,0.911222


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.880082,0.565640,0.046040,0.696774,0.772579,0.822321,0.572125,0.489072,0.229375,...,0.774739,0.925437,0.649030,0.959959,0.967547,0.852783,0.547811,0.142185,0.493564,0.898709
std,2812.670060,0.044536,0.125915,0.080256,0.106314,0.107068,0.145776,0.042381,0.070214,0.114863,...,0.088676,0.026616,0.033178,0.015922,0.016143,0.038170,0.310358,0.069758,0.258028,0.024344
min,39.000000,0.738347,0.351546,0.000135,0.300145,0.208113,0.126220,0.458608,0.387538,0.027038,...,0.598462,0.848885,0.575671,0.892453,0.912698,0.747202,0.002302,0.006965,0.057667,0.810008
25%,2572.000000,0.851341,0.460020,0.005412,0.645455,0.733269,0.782991,0.543124,0.436256,0.140192,...,0.702667,0.909972,0.624035,0.950982,0.958548,0.825940,0.247999,0.091493,0.267871,0.883058
50%,5093.000000,0.883157,0.546890,0.013182,0.710923,0.779755,0.877838,0.565614,0.467964,0.208173,...,0.769046,0.925099,0.646220,0.961938,0.969517,0.852233,0.641285,0.141098,0.464575,0.898051
75%,7482.000000,0.912487,0.670474,0.049429,0.769911,0.833687,0.913717,0.600468,0.525348,0.306769,...,0.852355,0.944839,0.671296,0.970488,0.979254,0.880145,0.820124,0.187412,0.702214,0.913975
max,9640.000000,0.978410,0.918417,0.541884,0.904069,0.996254,0.982072,0.717620,0.718218,0.588117,...,0.975305,0.987882,0.749267,0.994463,0.997522,0.954692,0.973299,0.353410,0.991496,0.964925
